# Génération d'image & jeu avec le gradient

L'idée est de récupérer une architecture pré-entrainée et d'aller jouer avec. Deux options assez simples sont envisageables:
1. Les approches *deepdream* : [github](https://github.com/google/deepdream/tree/master); [LeMonde](https://www.lemonde.fr/pixels/article/2015/07/09/on-a-teste-pour-vous-deep-dream-la-machine-a-reves-psychedeliques-de-google_4675562_4408996.html)
1. Les approches *style/content*, dites aussi transfert de style [github](https://github.com/Raed-Alshehri/Art_Generation_Neural_Style_Transfer). Si j'ai une image A qui m'interesse pour son contenu et une image B (i.e. un Van Gogh ou Munch) qui m'interesse pour son style, suis-je capable de construire une version A' qui prend le style de B?

Dans tous les cas, il faut récupérer un modèle (par exemple GoogLeNet) pré-entrainé puis récupérer et contraindre des couches intermédiaires (=définir une loss) puis ensuite activer le gradient sur l'entrée et laisser dériver l'image


In [ ]:
# importation d'un googlenet
# affichage des différentes couches => ce sera critique pour venir s'accrocher sur le réseau

import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)
model.eval()

## A. Classification d'image

Vérifier que le réseau marche bien en classification d'image

1. Charger une image
1. Appliquer le classifieur
1. Télécharger la signification des étiquettes

In [ ]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "img/dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms

# load image
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

In [ ]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

In [ ]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.imshow(input_image)

## B. Deepdream

1. Charger l'image + activer le gradient
1. La passer dans le réseau et récupérer une couche cachée
1. Définir une loss sur cette couche
1. Rétro-propager l'erreur

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import copy

# --- 2. Charger une image PIL et la prétraiter ---
def load_image(path, max_size=500):
    image = Image.open(path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize(max_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)


img = load_image("img/dog.jpg")
img_orig = copy.deepcopy(img)


On va utiliser les *hooks* = récupération très simplifiée d'information à l'intérieur du réseau
1. On *pose* le crochet
1. On fait le forward
1. On a l'information dans une structure de données, on va pouvoir créer notre loss!

In [ ]:

# --- 3. Choisir une couche cible ---
layer_name = 'inception4b'  # peut être 'inception3a', 'inception4d', 'inception4c' etc.

# --- 4. Extraire les activations intermédiaires ---
activations = {}

# Crochet
def hook_fn(module, input, output):
    activations['feat'] = output

# Récupérer le module voulu
target_layer = dict(model.named_modules())[layer_name]
hook = target_layer.register_forward_hook(hook_fn)

# --- 5. DeepDream : ascension de gradient sur l’image ---
img.requires_grad_(True)
optimizer = torch.optim.Adam([img], lr=0.1) # jouer avec le learning rate + nb itération

for i in range(30):  # nombre d’itérations
    optimizer.zero_grad()
    model(img)
    loss = -activations['feat'].norm()  # maximise la norme des activations ATTENTION à bien mettre un moins !
    loss.backward()
    optimizer.step()

    # Optionnel : normaliser légèrement les pixels pour la stabilité
    img.data = torch.clamp(img.data, -2, 2)

    if i % 5 == 0:
        print(f"Iteration {i}, Loss = {loss.item():.2f}")

hook.remove()


In [ ]:

# --- 6. Convertir l’image en format affichable ---
def deprocess(tensor):
    tensor = tensor.detach().cpu().squeeze(0)
    tensor = tensor * torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)
    tensor = tensor + torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    tensor = torch.clamp(tensor, 0, 1)
    return transforms.ToPILImage()(tensor)

dream_img = deprocess(img)
img_orig2  = deprocess(img_orig)
dream_img_c = dream_img.crop((50, 50, 200, 200))
img_orig_c = img_orig2.crop((50, 50, 200, 200))


plt.figure(figsize=(20,5))

plt.subplot(1,4,1)
plt.imshow(img_orig2)
plt.axis("off")

plt.subplot(1,4,2)
plt.imshow(dream_img)
plt.axis("off")

plt.subplot(1,4,3)
plt.imshow(img_orig_c)
plt.axis("off")

plt.subplot(1,4,4)
plt.imshow(dream_img_c)
plt.axis("off")


plt.show()


## C. Reconstruction d'image

1. A *encoder* (=forward) une image dans un réseau 
1. Isoler une représentation de cette image... Permettra-t-elle de reconstruire l'image?
1. Partir d'une image aléatoire (bruit blanc gaussien) avec gradient activé
1. Minimiser la distance au contenu encodé dans l'embedding

On va travailler avec le même réseau. Mais on introduit différentes optimisation pour aller plus vite en calcul.

In [ ]:
from torch.optim.lr_scheduler import OneCycleLR # pour accélérer


print("La version de torch est : ",torch.__version__)
print("Le calcul GPU est disponible ? ", torch.cuda.is_available())
# pour les possesseurs de mac M1 avec la dernière version de pytorch:
print("Le calcul GPU est disponible ? ", torch.backends.mps.is_available())

device = "mps" if torch.backends.mps.is_available() else "cpu"

Le transfert de style sera à faire par vous même... Mais afin de simplifier les choses, je vous propose un code permettant la reconstruction d'une image à partir d'un embedding

In [ ]:
img   = load_image("img/dog.jpg").to(device)

layer_content = 'inception3a'  # la première (bon encodage du contenu)
activations = {}

# Crochet
def hook_fn(module, input, output):
    activations['feat'] = output
target_layer = dict(model.named_modules())[layer_content]
hook = target_layer.register_forward_hook(hook_fn)


# target = une couche de l'encodage d'une image
with torch.no_grad():
    model(img)
    target = activations['feat'].clone()
    print("dimension de l'embedding : ", target.shape)


# Générer une image aléatoire
noise = torch.randn_like(img).to(device) #*0.1

# --- 5. Generation d'image : minimiser la norme entre les embeddings ---
noise.requires_grad_(True)

nepoch = 1000
max_lr=5e-1 # version optimisée OneCycle
optimizer = torch.optim.AdamW([noise], lr=max_lr)
scheduler = OneCycleLR(optimizer, max_lr=max_lr, steps_per_epoch=1, epochs=nepoch)

# optimizer = torch.optim.Adam([noise], lr=0.05) # jouer avec le learning rate + nb itération

for i in range(nepoch):  # nombre d’itérations
    optimizer.zero_grad()
    model(noise)
    loss = (target-activations['feat']).norm()  
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Optionnel : normaliser légèrement les pixels pour la stabilité
    noise.data = torch.clamp(noise.data, -2, 2)

    if i % 5 == 0:
        print(f"Iteration {i}, Loss = {loss.item():.2f}")

    # if i % 200 == 0:  # Si vous voulez voir la progression
    #     dream_img = deprocess(noise.to("cpu"))
    #     dream_img.save("save/image"+str(i)+".jpg")

hook.remove()


In [ ]:

# --- 6. Convertir l’image en format affichable ---

dream_img = deprocess(noise.to("cpu"))
img_orig2  = deprocess(img.to("cpu"))
dream_img_c = dream_img.crop((50, 50, 200, 200))
img_orig_c = img_orig2.crop((50, 50, 200, 200))


plt.figure(figsize=(20,5))

plt.subplot(1,4,1)
plt.imshow(img_orig2)
plt.axis("off")

plt.subplot(1,4,2)
plt.imshow(dream_img)
plt.axis("off")

plt.subplot(1,4,3)
plt.imshow(img_orig_c)
plt.axis("off")

plt.subplot(1,4,4)
plt.imshow(dream_img_c)
plt.axis("off")


plt.show()


## D. Transfert de style

Le transfert de style [lien](https://github.com/Raed-Alshehri/Art_Generation_Neural_Style_Transfer) consiste:
1. A *encoder* (=forward) deux images dans un réseau 
1. Isoler deux représentations à deux étages différents (couche intermédiaire pour le contenu, couche profonde pour le style)
1. Partir d'une image aléatoire (bruit blanc gaussien) avec gradient activé
1. Minimiser la distance à la fois au contenu et au style (= 2 normes à pondérer)

On va travailler avec le même réseau

A vous de jouer pour le transfert

In [ ]:
img   = load_image("img/dog.jpg").to(device)
style = load_image("img/vg.jpg").to(device)
model = model.to(device)

In [ ]:

# --- 3. Choisir une couche cible ---
layer_content = 'inception3a'  # la première
layer_style   = 'inception5a'  # la dernière

###  TODO 


In [ ]:

# --- 6. Convertir l’image en format affichable ---

dream_img = deprocess(noise.to("cpu"))
img_orig2  = deprocess(img.to("cpu"))
dream_img_c = dream_img.crop((50, 50, 200, 200))
img_orig_c = img_orig2.crop((50, 50, 200, 200))


plt.figure(figsize=(20,5))

plt.subplot(1,4,1)
plt.imshow(img_orig2)
plt.axis("off")

plt.subplot(1,4,2)
plt.imshow(dream_img)
plt.axis("off")

plt.subplot(1,4,3)
plt.imshow(img_orig_c)
plt.axis("off")

plt.subplot(1,4,4)
plt.imshow(dream_img_c)
plt.axis("off")


plt.show()


# Construction du sujet à partir de la correction

In [1]:
###  TODO )"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###